# Feature Engineering
This notebook will create a small set of robust, cross-dataset features derived from the canonical features.

## Goals
1. Normalize raw flow counts by time to capture traffic intensity (e.g., packets per second, bytes per second)
2. Capture flow density characteristics (e.g., average bytes per packet)
3. Encode directional asymmetry in traffic behavior (e.g., packet-level direction ratio, byte-level direction ratio)

In [1]:
!pip -q install "PyAthena[SQLAlchemy]" sqlalchemy s3fs

In [2]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

# Display settings
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Connect to Athena

In [3]:
sess = sagemaker.Session()
region = boto3.Session().region_name

results_bucket = sess.default_bucket()
athena_results_path = f"s3://{results_bucket}/athena/staging/"

database_name = "aai540_eda"

engine = create_engine(
    f"awsathena+rest://@athena.{region}.amazonaws.com:443/{database_name}",
    connect_args={"s3_staging_dir": athena_results_path, "region_name": region},
)
print("Region:", region)
print("Athena results:", athena_results_path)

Region: us-east-1
Athena results: s3://sagemaker-us-east-1-128131109986/athena/staging/


In [4]:
# Helper functions for queries
def exec_ddl(sql: str):
    with engine.begin() as conn:
        conn.execute(text(sql))

def read_sql(sql: str) -> pd.DataFrame:
    return pd.read_sql(sql, engine)

## Verify merged dataset

In [5]:
merged_dataset = read_sql(f"""
SELECT *
FROM {database_name}.merged_canonical_normalized
Limit 5
""")
merged_dataset

,duration,pkt_total,bytes_total,pkt_fwd,pkt_bwd,bytes_fwd,bytes_bwd,label,original_attack_type,attack_category,source_dataset
0,0.000127,2,0,1,1,0,0,1,dos,DoS/DDoS,TON_IoT
1,0.000001,2,0,1,1,0,0,1,dos,DoS/DDoS,TON_IoT
2,0.000010,2,0,1,1,0,0,1,dos,DoS/DDoS,TON_IoT
3,0.000066,2,0,1,1,0,0,1,dos,DoS/DDoS,TON_IoT
4,0.000020,2,0,1,1,0,0,1,dos,DoS/DDoS,TON_IoT


## Engineered Features
1. **`pkt_rate`** (*`pkt_rate = pkt_total / duration`*): Number of packets transmitted per second during the flow.
2. **`byte_rate`** (*`byte_rate = bytes_total / duration`*): Total bytes transferred per second during the flow.
3. **`bytes_per_pkt`** (*`bytes_per_pkt = bytes_total / (pkt_total + 1)`*): Average number of bytes carried per packet in the flow.
4. **`pkt_ratio`** (*`pkt_ratio = pkt_fwd / (pkt_bwd + 1)`*): Ratio of forward packets to backward packets in the flow.
5. **`byte_ratio`** (*`byte_ratio = bytes_fwd / (bytes_bwd + 1)`*): Ratio of forward bytes to backward bytes in the flow.

## **`pkt_rate`** (*`pkt_rate = pkt_total / duration`*)

This creates a new table version: `merged_canonical_normalized_v1`

In [7]:
# drop table if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v1")

# delete S3 data directory to avoid HIVE_PATH_ALREADY_EXISTS error
import subprocess
subprocess.run(["aws", "s3", "rm", f"s3://{results_bucket}/aai540/processed/merged_canonical_normalized_v1/", "--recursive"], check=False)

# create table pkt_rate
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v1
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v1/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN duration IS NULL OR duration <= 0 THEN NULL
    ELSE CAST(pkt_total AS DOUBLE) / CAST(duration AS DOUBLE)
  END AS pkt_rate
FROM {database_name}.merged_canonical_normalized
""")

In [8]:
read_sql(f"""
SELECT duration, pkt_total, pkt_rate
FROM {database_name}.merged_canonical_normalized_v1
pkt_total
Limit 25
""")

,duration,pkt_total,pkt_rate
0,0.165785,12,72.382906
1,0.000000,1,NaN
2,0.084254,9,106.819854
3,0.000000,1,NaN
4,0.087745,9,102.569947
5,0.165031,2,12.118935
6,0.073733,15,203.436724
7,0.000000,1,NaN
8,0.133657,11,82.300216
9,0.031547,3,95.096206


### Sanity check

In [9]:
read_sql(f"""
SELECT
  COUNT(*) AS rows_total,
  SUM(CASE WHEN duration IS NULL OR duration <= 0 THEN 1 ELSE 0 END) AS bad_duration_rows,
  SUM(CASE WHEN (duration IS NULL OR duration <= 0) AND pkt_rate IS NOT NULL THEN 1 ELSE 0 END) AS pkt_rate_should_be_null_but_isnt
FROM {database_name}.merged_canonical_normalized_v1
""")


,rows_total,bad_duration_rows,pkt_rate_should_be_null_but_isnt
0,26708942,5325854,0


## **`byte_rate`** (*`byte_rate = bytes_total / duration`*)

This creates a new table version: `merged_canonical_normalized_v2`

In [10]:
# drop v2 if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v2")

subprocess.run(["aws", "s3", "rm", f"s3://{results_bucket}/aai540/processed/merged_canonical_normalized_v2/", "--recursive"], check=False)


# Create v2 with byte_rate (built on v1 so pkt_rate is retained)
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v2
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v2/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN duration IS NULL OR duration <= 0 THEN NULL
    ELSE CAST(bytes_total AS DOUBLE) / CAST(duration AS DOUBLE)
  END AS byte_rate
FROM {database_name}.merged_canonical_normalized_v1
""")

In [11]:
read_sql(f"""
SELECT duration, bytes_total, byte_rate, pkt_total, pkt_rate
FROM {database_name}.merged_canonical_normalized_v2
WHERE duration IS NOT NULL
LIMIT 25
""")

,duration,bytes_total,byte_rate,pkt_total,pkt_rate
0,0.000142,0,0.0,2,14084.507042
1,31.723004,0,0.0,7,0.220660
2,0.000000,0,NaN,1,NaN
3,0.000452,0,0.0,2,4424.778761
4,0.001017,0,0.0,2,1966.568338
5,31.923626,0,0.0,5,0.156624
6,0.000448,0,0.0,2,4464.285714
7,0.000002,0,0.0,2,1000000.000000
8,0.000003,0,0.0,2,666666.666667
9,0.000047,0,0.0,2,42553.191489


### Sanity check

In [12]:
read_sql(f"""
SELECT
  COUNT(*) AS rows_total,
  SUM(CASE WHEN duration IS NULL OR duration <= 0 THEN 1 ELSE 0 END) AS bad_duration_rows,
  SUM(CASE WHEN (duration IS NULL OR duration <= 0) AND byte_rate IS NOT NULL THEN 1 ELSE 0 END) AS byte_rate_should_be_null_but_isnt
FROM {database_name}.merged_canonical_normalized_v2
""")


,rows_total,bad_duration_rows,byte_rate_should_be_null_but_isnt
0,26708942,5325854,0


## **`bytes_per_pkt`** (*`bytes_per_pkt = bytes_total / (pkt_total + 1)`*)

This creates a new table version: `merged_canonical_normalized_v3`

In [13]:
# drop v3 if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v3")

subprocess.run(["aws", "s3", "rm", f"s3://{results_bucket}/aai540/processed/merged_canonical_normalized_v3/", "--recursive"], check=False)

# create v3 with bytes_per_pkt (built on v2 so prior features are retained)
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v3
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v3/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN pkt_total IS NULL OR pkt_total <= 0 THEN NULL
    ELSE CAST(bytes_total AS DOUBLE) / CAST(pkt_total AS DOUBLE)
  END AS bytes_per_pkt
FROM {database_name}.merged_canonical_normalized_v2
""")

In [14]:
read_sql(f"""
SELECT duration, pkt_total, bytes_total, bytes_per_pkt
FROM {database_name}.merged_canonical_normalized_v3
WHERE bytes_total > 0
LIMIT 25
""")

,duration,pkt_total,bytes_total,bytes_per_pkt
0,1.793240,10,1528,152.800000
1,1.481577,1086,1822688,1678.349908
2,0.054140,5,842,168.400000
3,4.011152,3,108,36.000000
4,0.047653,3,844,281.333333
5,0.114414,7,822,117.428571
6,0.078947,10,2121,212.100000
7,0.107495,6,822,137.000000
8,0.471819,4,822,205.500000
9,6.031897,5,29,5.800000


### Sanity check

In [15]:
read_sql(f"""
SELECT
  COUNT(*) AS rows_total,
  SUM(CASE WHEN pkt_total IS NULL OR pkt_total <= 0 THEN 1 ELSE 0 END) AS bad_pkt_total_rows,
  SUM(CASE WHEN (pkt_total IS NULL OR pkt_total <= 0) AND bytes_per_pkt IS NOT NULL THEN 1 ELSE 0 END) AS bytes_per_pkt_should_be_null_but_isnt
FROM {database_name}.merged_canonical_normalized_v3
""")

,rows_total,bad_pkt_total_rows,bytes_per_pkt_should_be_null_but_isnt
0,26708942,115260,0


## **`pkt_ratio`** (*`pkt_ratio = pkt_fwd / (pkt_bwd + 1)`*)

This creates a new table version: `merged_canonical_normalized_v4`

In [16]:
# drop v4 if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v4")

subprocess.run(["aws", "s3", "rm", f"s3://{results_bucket}/aai540/processed/merged_canonical_normalized_v4/", "--recursive"], check=False)

# create v4 with pkt_ratio (built on v3 so prior features are retained)
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v4
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v4/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN pkt_fwd IS NULL OR pkt_fwd < 0 THEN NULL
    WHEN pkt_bwd IS NULL OR pkt_bwd < 0 THEN NULL
    ELSE CAST(pkt_fwd AS DOUBLE) / (CAST(pkt_bwd AS DOUBLE) + 1.0)
  END AS pkt_ratio
FROM {database_name}.merged_canonical_normalized_v3
""")

In [17]:
read_sql(f"""
SELECT duration, pkt_total, pkt_fwd, pkt_bwd, pkt_ratio
FROM {database_name}.merged_canonical_normalized_v4
LIMIT 25
""")

,duration,pkt_total,pkt_fwd,pkt_bwd,pkt_ratio
0,59.941705,25,6,19,0.300000
1,60.097945,5,3,2,1.000000
2,0.072395,2,1,1,0.500000
3,0.021884,2,1,1,0.500000
4,0.000144,3,2,1,1.000000
5,60.097884,3,2,1,1.000000
6,0.000004,3,2,1,1.000000
7,60.559121,4,3,1,1.500000
8,60.527756,4,3,1,1.500000
9,60.066318,5,3,2,1.000000


## **`byte_ratio`** (*`byte_ratio = bytes_fwd / (bytes_bwd + 1)`*)

This creates a new table version: `merged_canonical_normalized_v5`

In [18]:
# drop v5 if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v5")

subprocess.run(["aws", "s3", "rm", f"s3://{results_bucket}/aai540/processed/merged_canonical_normalized_v5/", "--recursive"], check=False)

# Create v5 with byte_ratio (built on v4 so prior features are retained)
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v5
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v5/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN bytes_fwd IS NULL OR bytes_fwd < 0 THEN NULL
    WHEN bytes_bwd IS NULL OR bytes_bwd < 0 THEN NULL
    ELSE CAST(bytes_fwd AS DOUBLE) / (CAST(bytes_bwd AS DOUBLE) + 1.0)
  END AS byte_ratio
FROM {database_name}.merged_canonical_normalized_v4
""")

In [19]:
read_sql(f"""
SELECT
  bytes_fwd,
  bytes_bwd,
  byte_ratio
FROM {database_name}.merged_canonical_normalized_v5
LIMIT 25
""")

,bytes_fwd,bytes_bwd,byte_ratio
0,0,0,0.0
1,0,0,0.0
2,0,0,0.0
3,0,0,0.0
4,0,0,0.0
5,0,0,0.0
6,0,0,0.0
7,0,0,0.0
8,0,0,0.0
9,0,0,0.0


## Finalize Feature Engineering Table

In [22]:
# drop final table if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.feature_engineered")

subprocess.run(["aws", "s3", "rm", f"s3://{results_bucket}/aai540/processed/feature_engineered/", "--recursive"], check=False)

# create final feature_engineered table from v5
exec_ddl(f"""
CREATE TABLE {database_name}.feature_engineered
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/feature_engineered/',
  parquet_compression = 'SNAPPY'
) AS
SELECT *
FROM {database_name}.merged_canonical_normalized_v5
""")


In [25]:
read_sql(f"""
SELECT
  *
FROM {database_name}.feature_engineered
LIMIT 10
""")

,duration,pkt_total,bytes_total,pkt_fwd,pkt_bwd,bytes_fwd,bytes_bwd,label,original_attack_type,attack_category,source_dataset,pkt_rate,byte_rate,bytes_per_pkt,pkt_ratio,byte_ratio
0,60.704337,5,0,3,2,0,0,1,ddos,DoS/DDoS,TON_IoT,0.082366,0.000000,0.0,1.000000,0.000000
1,22.517201,9,0,5,4,0,0,1,ddos,DoS/DDoS,TON_IoT,0.399694,0.000000,0.0,1.000000,0.000000
2,0.002026,2,78,1,1,39,39,1,ddos,DoS/DDoS,TON_IoT,987.166831,38499.506417,39.0,0.500000,0.975000
3,0.011849,10,296,6,4,125,171,1,ddos,DoS/DDoS,TON_IoT,843.953076,24981.011056,29.6,1.200000,0.726744
4,0.002780,2,78,1,1,39,39,1,ddos,DoS/DDoS,TON_IoT,719.424460,28057.553957,39.0,0.500000,0.975000
5,0.003027,2,86,1,1,43,43,1,ddos,DoS/DDoS,TON_IoT,660.720185,28410.967955,43.0,0.500000,0.977273
6,60.587032,23,2277,7,16,486,1791,1,ddos,DoS/DDoS,TON_IoT,0.379619,37.582300,99.0,0.411765,0.271205
7,21.719092,9,0,5,4,0,0,1,ddos,DoS/DDoS,TON_IoT,0.414382,0.000000,0.0,1.000000,0.000000
8,0.007780,10,383,6,4,212,171,1,ddos,DoS/DDoS,TON_IoT,1285.347044,49228.791774,38.3,1.200000,1.232558
9,60.701095,5,0,3,2,0,0,1,ddos,DoS/DDoS,TON_IoT,0.082371,0.000000,0.0,1.000000,0.000000


## Data integrity check

In [26]:
profile_invalid = read_sql(f"""
SELECT
  COUNT(*) AS total_rows,

  -- NULL counts
  SUM(CASE WHEN duration  IS NULL THEN 1 ELSE 0 END) AS duration_null,
  SUM(CASE WHEN pkt_total IS NULL THEN 1 ELSE 0 END) AS pkt_total_null,
  SUM(CASE WHEN bytes_total IS NULL THEN 1 ELSE 0 END) AS bytes_total_null,
  SUM(CASE WHEN pkt_fwd   IS NULL THEN 1 ELSE 0 END) AS pkt_fwd_null,
  SUM(CASE WHEN pkt_bwd   IS NULL THEN 1 ELSE 0 END) AS pkt_bwd_null,
  SUM(CASE WHEN bytes_fwd IS NULL THEN 1 ELSE 0 END) AS bytes_fwd_null,
  SUM(CASE WHEN bytes_bwd IS NULL THEN 1 ELSE 0 END) AS bytes_bwd_null,

  -- Negative counts
  SUM(CASE WHEN duration  < 0 THEN 1 ELSE 0 END) AS duration_neg,
  SUM(CASE WHEN pkt_total < 0 THEN 1 ELSE 0 END) AS pkt_total_neg,
  SUM(CASE WHEN bytes_total < 0 THEN 1 ELSE 0 END) AS bytes_total_neg,
  SUM(CASE WHEN pkt_fwd   < 0 THEN 1 ELSE 0 END) AS pkt_fwd_neg,
  SUM(CASE WHEN pkt_bwd   < 0 THEN 1 ELSE 0 END) AS pkt_bwd_neg,
  SUM(CASE WHEN bytes_fwd < 0 THEN 1 ELSE 0 END) AS bytes_fwd_neg,
  SUM(CASE WHEN bytes_bwd < 0 THEN 1 ELSE 0 END) AS bytes_bwd_neg,

  -- Zero counts
  SUM(CASE WHEN duration  = 0 THEN 1 ELSE 0 END) AS duration_zero,
  SUM(CASE WHEN pkt_total = 0 THEN 1 ELSE 0 END) AS pkt_total_zero,
  SUM(CASE WHEN bytes_total = 0 THEN 1 ELSE 0 END) AS bytes_total_zero

FROM {database_name}.feature_engineered
""")
profile_invalid

,total_rows,duration_null,pkt_total_null,bytes_total_null,pkt_fwd_null,pkt_bwd_null,bytes_fwd_null,bytes_bwd_null,duration_neg,pkt_total_neg,bytes_total_neg,pkt_fwd_neg,pkt_bwd_neg,bytes_fwd_neg,bytes_bwd_neg,duration_zero,pkt_total_zero,bytes_total_zero
0,26708942,0,0,0,0,0,0,0,115,0,0,0,0,0,0,5325739,115260,15981998


## Count rows to be dropped

In [27]:
invalid_row_summary = read_sql(f"""
SELECT
  COUNT(*) AS total_rows,
  SUM(
    CASE WHEN
      -- NULL raw features
      duration IS NULL OR pkt_total IS NULL OR bytes_total IS NULL OR
      pkt_fwd IS NULL OR pkt_bwd IS NULL OR bytes_fwd IS NULL OR bytes_bwd IS NULL OR

      -- negative raw features
      duration < 0 OR pkt_total < 0 OR bytes_total < 0 OR
      pkt_fwd < 0 OR pkt_bwd < 0 OR bytes_fwd < 0 OR bytes_bwd < 0 OR

      -- zeros 
      duration = 0 OR pkt_total = 0
    THEN 1 ELSE 0 END
  ) AS rows_to_drop
FROM {database_name}.feature_engineered
""")

invalid_row_summary

,total_rows,rows_to_drop
0,26708942,5325854


## Create cleaned table (drop ALL invalid rows: NULL, negative, or zero)

In [29]:
# create a cleaned table
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.feature_engineered_cleaned")

subprocess.run(["aws", "s3", "rm", f"s3://{results_bucket}/aai540/processed/feature_engineered_cleaned/", "--recursive"], check=False)

exec_ddl(f"""
CREATE TABLE {database_name}.feature_engineered_cleaned
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/feature_engineered_cleaned/',
  parquet_compression = 'SNAPPY'
) AS
SELECT *
FROM {database_name}.feature_engineered
WHERE
  -- Raw canonical must exist
  duration IS NOT NULL
  AND pkt_total IS NOT NULL
  AND bytes_total IS NOT NULL
  AND pkt_fwd IS NOT NULL
  AND pkt_bwd IS NOT NULL
  AND bytes_fwd IS NOT NULL
  AND bytes_bwd IS NOT NULL

  -- Raw canonical must be non-negative
  AND duration >= 0
  AND pkt_total >= 0
  AND bytes_total >= 0
  AND pkt_fwd >= 0
  AND pkt_bwd >= 0
  AND bytes_fwd >= 0
  AND bytes_bwd >= 0

  -- Drop zero denominators
  AND duration > 0
  AND pkt_total > 0
""")

## Inspect 

In [30]:
invalid_rows_sample = read_sql(f"""
SELECT
  duration, pkt_total, bytes_total, pkt_fwd, pkt_bwd, bytes_fwd, bytes_bwd,
  pkt_rate, byte_rate, bytes_per_pkt, pkt_ratio, byte_ratio,
  label, source_dataset
FROM {database_name}.feature_engineered_cleaned
WHERE
  duration IS NULL OR pkt_total IS NULL OR bytes_total IS NULL OR
  pkt_fwd IS NULL OR pkt_bwd IS NULL OR bytes_fwd IS NULL OR bytes_bwd IS NULL OR
  duration < 0 OR pkt_total < 0 OR bytes_total < 0 OR
  pkt_fwd < 0 OR pkt_bwd < 0 OR bytes_fwd < 0 OR bytes_bwd < 0 OR
  duration = 0 OR pkt_total = 0
LIMIT 20
""")

invalid_rows_sample

,duration,pkt_total,bytes_total,pkt_fwd,pkt_bwd,bytes_fwd,bytes_bwd,pkt_rate,byte_rate,bytes_per_pkt,pkt_ratio,byte_ratio,label,source_dataset


## Clean Athena catalog

In [31]:
tables_to_drop = [
    "merged_canonical_normalized_v1",
    "merged_canonical_normalized_v2",
    "merged_canonical_normalized_v3",
    "merged_canonical_normalized_v4",
    "merged_canonical_normalized_v5",
]

for t in tables_to_drop:
    exec_ddl(f"DROP TABLE IF EXISTS {database_name}.{t}")